In [23]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium

In [187]:
df = pd.read_html('https://en.wikipedia.org/wiki/Quarters_of_Paris')

In [202]:
df_Paris = df[0]

In [203]:
df_Paris= df_Paris.drop(['Population in1999[3]','Area(hectares)[3]','Map'],axis = 1)
df_Paris.columns = ['Arrond','Quartier Num','Quartier Name']

In [204]:
df_Paris["Quartier Name"] = df_Paris["Quartier Name"]  + " Paris"
df_Paris

,Arrond,Quartier Num,Quartier Name
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois Paris
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles Paris
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal Paris
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme Paris
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon Paris
...,...,...,...
75,"19th arrondissement(Called ""des Buttes-Chaumont"")",76th,Combat Paris
76,"20th arrondissement(Called ""de Ménilmontant"")",77th,Belleville Paris
77,"20th arrondissement(Called ""de Ménilmontant"")",78th,Saint-Fargeau Paris
78,"20th arrondissement(Called ""de Ménilmontant"")",79th,Père-Lachaise Paris


In [207]:
latitudes = []
longitudes = []
for i in df_Paris['Quartier Name']:
    address = i
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(i)
    latitude = location.latitude
    longitude = location.longitude
    latitudes.append(latitude)
    longitudes.append(longitude)

In [208]:
df_Paris['latitude'] = latitudes
df_Paris['longitude'] =longitudes
df_Paris

,Arrond,Quartier Num,Quartier Name,latitude,longitude
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois Paris,48.860211,2.336299
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles Paris,48.862466,2.346009
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal Paris,48.863585,2.336204
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme Paris,48.867463,2.329428
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon Paris,48.869135,2.332909
...,...,...,...,...,...
75,"19th arrondissement(Called ""des Buttes-Chaumont"")",76th,Combat Paris,48.877421,2.371020
76,"20th arrondissement(Called ""de Ménilmontant"")",77th,Belleville Paris,48.871727,2.385085
77,"20th arrondissement(Called ""de Ménilmontant"")",78th,Saint-Fargeau Paris,48.870362,2.406736
78,"20th arrondissement(Called ""de Ménilmontant"")",79th,Père-Lachaise Paris,48.861217,2.393929


In [209]:
def getlocator(address):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address['Quartier Name'])
    latitude = location.latitude
    longitude = location.longitude
    print ("{}'s latitude is {} and longitude is {}".format(address,latitude,longitude))
    return latitude, longitude

In [210]:
CLIENT_ID = 'NEYENBBKMAHNZ1QA5XLNFX34C1JVRGNDHV55O2EXSMK0CCLN' # your Foursquare ID
CLIENT_SECRET = 'UNU30MQEGVP01QQAXPYYI55HPWAXU1DVNPSOM5SXIU11VIIJ' # your Foursquare Secret
VERSION = '202101020' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NEYENBBKMAHNZ1QA5XLNFX34C1JVRGNDHV55O2EXSMK0CCLN
CLIENT_SECRET:UNU30MQEGVP01QQAXPYYI55HPWAXU1DVNPSOM5SXIU11VIIJ


In [211]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Quartier Name', 
                  'Quartier Latitude', 
                  'Quartier Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [212]:
Paris_venues = getNearbyVenues(df_Paris['Quartier Name'], df_Paris['latitude'], df_Paris['longitude'], radius=500)

Saint-Germain-l'Auxerrois Paris
Les Halles Paris
Palais-Royal Paris
Place-Vendôme Paris
Gaillon Paris
Vivienne Paris
Mail Paris
Bonne-Nouvelle Paris
Arts-et-Métiers Paris
Enfants-Rouges Paris
Archives Paris
Sainte-Avoye Paris
Saint-Merri Paris
Saint-Gervais Paris
Arsenal Paris
Notre-Dame Paris
Quartier Saint-Victor Paris
Jardin-des-Plantes Paris
Val-de-Grâce Paris
Sorbonne Paris
Monnaie Paris
Odéon Paris
Notre-Dame-des-Champs Paris
Saint-Germain-des-Prés Paris
Saint-Thomas-d'Aquin Paris
Les Invalides Paris
École-Militaire Paris
Gros-Caillou Paris
Champs-Élysées Paris
Faubourg-du-Roule Paris
La Madeleine Paris
Europe Paris
Saint-Georges Paris
Chaussée-d'Antin Paris
Faubourg-Montmartre Paris
Rochechouart Paris
Saint-Vincent-de-Paul Paris
Porte-Saint-Denis Paris
Porte-Saint-Martin Paris
Hôpital-Saint-Louis Paris
Folie-Méricourt Paris
Saint-Ambroise Paris
La Roquette Paris
Sainte-Marguerite Paris
Bel-Air Paris
Picpus Paris
Bercy Paris
Quinze-Vingts Paris
Salpêtrière Paris
La Gare Paris
Mai

In [213]:
Paris_venues.shape

(5416, 7)

In [214]:
Paris_venues.groupby("Quartier Name").count()

,Quartier Latitude,Quartier Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Quartier Name,,,,,,
Amérique Paris,13,13,13,13,13,13
Archives Paris,100,100,100,100,100,100
Arsenal Paris,71,71,71,71,71,71
Arts-et-Métiers Paris,100,100,100,100,100,100
Auteuil Paris,51,51,51,51,51,51
...,...,...,...,...,...,...
Sorbonne Paris,100,100,100,100,100,100
Val-de-Grâce Paris,54,54,54,54,54,54
Vivienne Paris,97,97,97,97,97,97


In [215]:
len(Paris_venues['Venue Category'].unique())

291

In [236]:
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Quartier Name'] = Paris_venues['Quartier Name'] 

# move neighborhood column to the first column
fixed_columns = [Paris_onehot.columns[-1]] + list(Paris_onehot.columns[:-1])
Paris_onehot = Paris_onehot[fixed_columns]

Paris_onehot.head()

,Quartier Name,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Saint-Germain-l'Auxerrois Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Saint-Germain-l'Auxerrois Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Saint-Germain-l'Auxerrois Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Saint-Germain-l'Auxerrois Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Saint-Germain-l'Auxerrois Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [237]:
Paris_grouped = Paris_onehot.groupby('Quartier Name').mean().reset_index()

In [238]:
#Restaurant = Paris_grouped[[col for col in Paris_grouped.columns if 'Restaurant' in col]].sum(axis=1)

#Paris_grouped = Paris_grouped.drop([col for col in Paris_grouped.columns if 'Restaurant' in col], axis = 1)

#Paris_grouped['Restaurant'] = Restaurant

In [239]:
Paris_grouped

,Quartier Name,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Amérique Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Archives Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.010000,0.0,0.0,0.00,0.010000,0.000000,0.000000,0.0,0.0,0.0
2,Arsenal Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.028169,0.0,0.0,0.00,0.014085,0.000000,0.000000,0.0,0.0,0.0
3,Arts-et-Métiers Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.01,...,0.020000,0.0,0.0,0.03,0.030000,0.020000,0.000000,0.0,0.0,0.0
4,Auteuil Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.000000,0.039216,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Sorbonne Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.01,0.010000,0.000000,0.000000,0.0,0.0,0.0
76,Val-de-Grâce Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.018519,0.000000,0.000000,0.0,0.0,0.0
77,Vivienne Paris,0.0,0.0,0.0,0.0,0.0,0.010309,0.010309,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.051546,0.010309,0.010309,0.0,0.0,0.0
78,École-Militaire Paris,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [240]:
Paris_grouped_clustering = Paris_grouped.drop('Quartier Name', 1)
num_clusters = 6
k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(Paris_grouped_clustering)
labels = k_means.labels_
labels

array([2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 1, 1, 2, 0, 2, 2, 0, 0, 1, 0, 1, 2,
       0, 5, 2, 1, 1, 2, 0, 2, 4, 0, 0, 0, 2, 0, 0, 1, 1, 0, 5, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 2, 2, 2, 1, 2, 3, 2, 0,
       0, 0, 2, 0, 1, 4, 0, 2, 0, 0, 2, 0, 1, 2], dtype=int32)

In [241]:
Paris_grouped.insert(0, 'Cluster Labels', labels)

In [242]:
Paris_Map = pd.merge(Paris_grouped.iloc[:,0:2],df_Paris,on="Quartier Name")

In [243]:
Paris_Map.sort_values('Cluster Labels')

,Cluster Labels,Quartier Name,Arrond,Quartier Num,latitude,longitude
39,0,Mail Paris,"2nd arrondissement(Called ""de la Bourse"")",7th,48.868054,2.344593
65,0,Saint-Germain-des-Prés Paris,"6th arrondissement(Called ""du Luxembourg"")",24th,48.854429,2.333707
56,0,Porte-Saint-Denis Paris,"10th arrondissement(Called ""de l'Entrepôt"")",38th,48.869793,2.352686
31,0,La Gare Paris,"13th arrondissement(Called ""des Gobelins"")",50th,48.827898,2.372877
32,0,La Madeleine Paris,"8th arrondissement(Called ""de l'Élysée"")",31st,48.870137,2.324562
...,...,...,...,...,...,...
63,3,Saint-Fargeau Paris,"20th arrondissement(Called ""de Ménilmontant"")",78th,48.870362,2.406736
71,4,Saint-Vincent-de-Paul Paris,"10th arrondissement(Called ""de l'Entrepôt"")",37th,48.880065,2.356772
30,4,La Chapelle Paris,"18th arrondissement(Called ""des Buttes-Montmar...",72nd,48.884407,2.360306
23,5,Goutte-d'Or Paris,"18th arrondissement(Called ""des Buttes-Montmar...",71st,48.892676,2.356040


In [244]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode("Paris")
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_Map['latitude'], Paris_Map['longitude'], Paris_Map['Quartier Name'], Paris_Map['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters